# Import các thư vện cần thiết

In [1]:
import os
import pandas as pd
import cloudscraper
from bs4 import BeautifulSoup

# Khởi tạo Cloudscraper (giả lập trình duyệt để vượt qua Cloudflare)

In [ ]:
scraper = cloudscraper.create_scraper()

base_url = "https://www.kraken.com/vi-vn/prices?page="
all_data = []

# Lặp qua từng trang

In [ ]:
for page in range(1, 5):  
    url = base_url + str(page)
    print(f"🔄 Đang tải trang {url}...")

    # Gửi request đến trang web
    response = scraper.get(url)

    # Kiểm tra nếu request bị chặn
    if response.status_code != 200:
        print(f"❌ Lỗi {response.status_code} khi tải trang {url}")
        continue  # Bỏ qua trang lỗi và thử trang tiếp theo

    # Phân tích HTML với BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Tìm tất cả các hàng dữ liệu <tr>
    rows = soup.find_all("tr", class_="fc150")
    
    for row in rows:
        cells = row.find_all("td")
        
        try:
            rank = cells[0].get_text(strip=True)  # Xếp hạng
            name = cells[1].find("span", class_="fc-bc02027a-6").get_text(strip=True)  # Tên coin
            symbol = cells[1].find("span", class_="fc-bc02027a-7").get_text(strip=True)  # Ký hiệu coin
            price = cells[2].get_text(strip=True)  # Giá hiện tại
            change_24h = cells[4].get_text(strip=True)  # Thay đổi 24h
            market_cap = cells[5].get_text(strip=True)  # Vốn hóa thị trường
            
            all_data.append([rank, name, symbol, price, change_24h, market_cap])
        
        except IndexError:
            continue  # Bỏ qua hàng lỗi nếu không đủ cột

# Tạo DataFrame và Lưu file CSV

In [ ]:
columns = ["Xếp hạng", "Tên", "Mã", "Giá hiện tại", "Thay đổi 24h", "Vốn hóa thị trường"]
df = pd.DataFrame(all_data, columns=columns)

csv_path = os.path.join(os.getcwd(), "kraken_all_pages.csv")
df.to_csv(csv_path, index=False, encoding="utf-8-sig")

print(f"✅ Dữ liệu đã được lưu vào {csv_path}")

# Xuất ra một vài dữ liệu đầu tiên dưới dạng bảng

In [ ]:
import pandas as pd
df = pd.read_csv("kraken_all_pages.csv", sep=",")
from IPython.display import display
display(df)

,Xếp hạng,Tên,Mã,Giá hiện tại,Thay đổi 24h,Vốn hóa thị trường
0,1,Bitcoin,BTC,"92.062,00 $","+5,41%","1,82 NT $"
1,2,Ethereum,ETH,"2.287,21 $","+5,03%","275,20 T $"
2,3,XRP,XRP,"2,53 $","+0,81%","146,16 T $"
3,4,Tether,USDT,"1,00 $","+0,03%","142,72 T $"
4,5,Solana,SOL,"148,77 $","+3,08%","75,45 T $"
...,...,...,...,...,...,...
352,353,Sky,SKY,"0,057 $","-1,11%",-
353,354,Multichain,MULTI,"0,84 $","-1,92%",-
354,355,Fantom,FTM,"0,59 $","+7,33%",-
355,356,Crust Shadow,CSM,"0,0054 $","+22,55%",-
